In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import sys
import re
import os
from pathlib import Path
from functools import partial


import blpapi
# import clarion
import ipywidgets as w
import ipyvuetify as v
import numpy as np
import pandas as pd
import QuantLib as ql
from IPython.display import display
from ipydatagrid import DataGrid, TextRenderer, BarRenderer, Expr
from bqplot import LinearScale, ColorScale, OrdinalColorScale, OrdinalScale
# from clarion import positions

from api.gui.base import View
from api.blpw import BlpQuery
from api.data.base import data_path
from api.data.curves import curves
from api.data.utils import add_pcs, parse_tenor
from api.pricing.cds import get_isda_cds
from api.data.processing import apply_function
from api.pricing.utils import (
    get_cds_specs,
    get_cds_curve,
    get_cds_spreads,
    get_risk_free_curve,
    get_cds_implied_price,
    get_curve,
)

In [ ]:
dt_fmt = '%Y-%m-%d'
bbg_dt_fmt = '%Y%m%d'

In [ ]:
bq = BlpQuery(timeout=50000).start()
self = bq

In [ ]:
class Comp(View):
    
    def make_widgets(self):
        paste_btn = v.Btn(
            fab=True,
            # dark=True,
            small=True,
            plain=True,
            class_='mx-0',
            children=[v.Icon(children=['mdi-content-paste'])]
        )

        refresh_btn = v.Btn(
            fab=True,
            # dark=True,
            small=True,
            plain=True,
            class_='mx-2',
            children=[v.Icon(children=['mdi-sync'])]
        )

        pcs_tf = v.TextField(
            v_model=None,
            outlined=True,
            dense=True,
            label='pcs',
            clearable=True
        )
        self.paste_btn = paste_btn
        self.refresh_btn = refresh_btn
        self.pcs_tf = pcs_tf
        
        self.out = w.Output()
    
    def make_view(self):
        
        paste_btn = self.paste_btn
        refresh_btn = self.refresh_btn
        pcs_tf = self.pcs_tf
        out = self.out
        
        param_box = v.Container(
            children=[
                v.Row(
                    children=[
                        v.Col(
                            cols=2,
                            children=[pcs_tf],
                            class_="my-0 py-0"
                        ),
                        v.Col(
                            cols=2,
                            children=[paste_btn, refresh_btn],
                            class_="my-0 py-0"
                        ),

                    ],
                    align_content='center',
                ),
            ]
        )
        
        view = w.VBox(
            children=[
                param_box,
                out,
            ]
        )

        self.param_box = param_box
        self.view = view
    
    def link(self):
        paste_btn = self.paste_btn
        refresh_btn = self.refresh_btn
        pcs_tf = self.pcs_tf
        out = self.out
        
        paste_btn.on_event(
            'click', 
            partial(on_click_paste, self=self)
        )

        refresh_btn.on_event(
            'click', 
            partial(
                out.capture(clear_output=True)(on_click_refresh_grid), 
                self=self
            )
        )

In [ ]:
def get_securities(securities, id_type='isin', pcs=None, sep=None):
    if pcs:
        if id_type.lower() == 'isin':
            sep = '@'
        else: 
            sep = ' '
        securities = add_pcs(securities, source=pcs, sep=sep)
    
    return [f'/{id_type}/{sec}' for sec in securities]

In [ ]:
def millify(cell):
    n = abs(float(cell.value))
    
    if n < 1000:
        return format(cell.value, '.3f') 
    elif n < 1000000:
        return format(cell.value / 1000, '.3f') + 'k'
    elif n < 1000000000:
        return format(cell.value / 1000000, '.3f') + 'm'
    elif n < 1000000000000:
        return format(cell.value / 1000000000, '.3f') + 'b'
    else:
        return format(cell.value / 1000000000000, '.3f') + 't'

    
def millify0(cell):
    n = abs(float(cell.value))
    
    if n < 1000:
        return format(cell.value, '.0f') 
    elif n < 1000000:
        return format(cell.value / 1000, '.0f') + 'k'
    elif n < 1000000000:
        return format(cell.value / 1000000, '.0f') + 'm'
    elif n < 1000000000000:
        return format(cell.value / 1000000000, '.0f') + 'b'
    else:
        return format(cell.value / 1000000000000, '.0f') + 't'

    
def get_millify(decimal=3):
    def millify(cell):
        n = abs(float(cell.value))

        if n < 1000:
            return format(cell.value, ".{}f".format(decimal)) 
        elif n < 1000000:
            return format(cell.value / 1000, ".{}f".format(decimal)) + 'k'
        elif n < 1000000000:
            return format(cell.value / 1000000, ".{}f".format(decimal)) + 'm'
        elif n < 1000000000000:
            return format(cell.value / 1000000000, ".{}f".format(decimal)) + 'b'
        else:
            return format(cell.value / 1000000000000, ".{}f".format(decimal)) + 't'
    return millify


def text_color(cell):
    pass

In [ ]:
def bbg_comp(data_, pcs=None):
    securities = get_securities(data_['ISIN'], pcs=pcs)
    fields = [
        'px_last',
    ]
    
    bbg_data = bq.bdp(
        securities=securities,
        fields=fields
    )
    bbg_data['security'] = bbg_data['security'].replace(data_['ISIN'].set_axis(securities))
    comp_data = data_.merge(bbg_data, how='left', left_on='ISIN', right_on='security').drop('security', axis=1)
    comp_data['price_diff'] = (comp_data['Quoted Price'] - comp_data['px_last'])
    comp_data['notional_diff'] = (comp_data['price_diff'] * comp_data['Notional']) / 100
    comp_data['real pnl'] = comp_data['Total'] + comp_data['notional_diff']
    comp_data = comp_data.sort_values('notional_diff', ascending=False)
    
    rename = {
        'Total': 'clarion pnl',
        'Quoted Price': 'clarion price',
        'px_last': 'bloomberg price',
        'notional_diff': 'adjustment'
    }
    
    return comp_data.rename(rename, axis=1)

In [ ]:
def make_datagrid(comp_data):
    renderers = {

        'Notional': TextRenderer(
            text_value=Expr(millify),
            text_color=Expr('"red" if cell.value < 0 else default_value')
        ),
        'clarion pnl': TextRenderer(
            text_value=Expr(millify0),
            text_color=Expr('"red" if cell.value < 0 else default_value')
        ),
        'adjustment': BarRenderer(
            text_value=Expr(millify),
            text_color=Expr('"red" if cell.value < 0 else default_value'),
            horizontal_alignment="center",
            bar_color=ColorScale(min=comp_data['adjustment'].min(), max=comp_data['adjustment'].max()),
            bar_value=LinearScale(min=comp_data['adjustment'].min(), max=comp_data['adjustment'].max())
        ),
        'price_diff': BarRenderer(
            text_value=Expr("format(cell.value, '.2f')"),
            text_color=Expr('"red" if cell.value < 0 else default_value'),
            horizontal_alignment="center",
            bar_color=ColorScale(min=comp_data['price_diff'].min(), max=comp_data['price_diff'].max()),
            bar_value=LinearScale(min=comp_data['price_diff'].min(), max=comp_data['price_diff'].max())
        ),
        'real pnl': TextRenderer(
            text_value=Expr(millify0),
            text_color=Expr('"red" if cell.value < 0 else default_value')
        ),
    }
    
    dg = DataGrid(
        comp_data, 
        renderers=renderers, 
        base_column_size=80,
        editable=False,
        selection_mode='cell',
        column_widths={
            'Description': 150, 
            'Issuer': 80, 
            'bloomberg price': 120,
            'Ccy': 60,
            'adjustment': 90,
            'clarion pnl': 90,
            'clarion price': 95
        }
        
    )
    # dg.auto_fit_columns = True
    return dg

def make_datagrid2(comp_data):
    renderers = {

        'Notional': TextRenderer(
            text_value=Expr(millify),
            text_color=Expr('"red" if cell.value < 0 else default_value')
        ),
        'clarion pnl': TextRenderer(
            text_value=Expr(millify0),
            text_color=Expr('"red" if cell.value < 0 else default_value')
        ),
        'adjustment': BarRenderer(
            text_value=Expr(millify),
            text_color=Expr('"red" if cell.value < 0 else default_value'),
            horizontal_alignment="center",
            bar_color=ColorScale(min=comp_data['adjustment'].min(), max=comp_data['adjustment'].max()),
            bar_value=LinearScale(min=comp_data['adjustment'].min(), max=comp_data['adjustment'].max())
        ),
        'real pnl': TextRenderer(
            text_value=Expr(millify0),
            text_color=Expr('"red" if cell.value < 0 else default_value')
        ),
    }
    
    dg = DataGrid(
        comp_data, 
        renderers=renderers, 
        base_column_size=80,
        editable=False,
        selection_mode='cell',
        column_widths={
            'Issuer': 80, 
            'Ccy': 60,
            'clarion pnl': 90,
            'clarion price': 95,
            'adjustment': 90,
        }
        
    )
    # dg.auto_fit_columns = True
    return dg


In [ ]:
class N:
    def __init__(self, n):
        self.value = n 

In [ ]:
def parse_clipboard():
    data = pd.read_clipboard(sep='\t', header=[1])
    data['Notional'] = data['Notional'].str.replace(',', '').astype(float)
    cols = [
        'Issuer', 
        'Trade Type',
        'Description',
        'Ccy',
        'Expiry/ Maturity',
        'ISIN',
        'Total',
        'Notional',
        'Quoted Price',
    ]
    
    data_ = data[cols].dropna(subset='ISIN')
    data_ = data_.drop('Notional', axis=1)\
        .drop_duplicates(subset='ISIN')\
        .merge(data_.groupby('ISIN', as_index=False, sort=False)['Notional'].sum(), on='ISIN')
    data_ = data_[[*data_.columns[:-2], data_.columns[-1], data_.columns[-2]]]
    data_ = data_.loc[data_['Notional'] != 0, :]
    return data_

In [ ]:
def on_click_paste(widget, event, data, self):
    try:
        self.data_ = parse_clipboard()
    except Exception as e:
        widget.color = 'error'
    else:
        widget.color = 'success'
    finally:
        widget.loading = False

In [ ]:
def on_click_refresh_grid(widget, event, data, self):
    
    widget.loading = True
    try:
        data_ = self.data_
        pcs = self.pcs_tf.v_model
        comp_data = bbg_comp(data_, pcs)
        self.comp_data = comp_data
        dg = make_datagrid(comp_data)
        html = w.HTML(f"<b>The total adjustment is: {get_millify(2)(N(comp_data['adjustment'].sum()))} <b>")
        display(html)
        display(w.HTML(f"<br> <b> Breakdown by Instrument <b>"))
        self.dg = dg
        display(dg)
        agg_data = self.comp_data.groupby(['Issuer', 'Ccy'], sort=False, as_index=False)\
            [['Notional', 'adjustment', 'clarion pnl', 'real pnl']].sum().sort_values('adjustment', ascending=False)
        self.agg_data = agg_data
        dg2 = make_datagrid2(agg_data)
        self.dg2 = dg2
        display(w.HTML(f"<br> <b> Breakdown by Issuer <b>"))
        display(dg2)
        # box = w.HBox([dg, dg2])
    except Exception as e:
        widget.color = 'error'
        raise e
    finally:
        widget.loading = False

In [206]:
self = Comp()
self